In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

from tuneavideo.models.unet_with_controlnet import UNet2DConditionModel
from tuneavideo.data.dataset import TuneAVideoMoreShotDataset
from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.pipelines.pipeline_tuneavideo_controlnet import TuneAVideoControlNetPipeline
from diffusers import StableDiffusionPipeline
from tuneavideo.models.controlnet_blocks import InflatedControlNetInputHintBlock3D, ControlNetInputHintBlock

import torch
from transformers import CLIPTokenizer
import numpy as np
from einops import rearrange
from PIL import Image
from glob import glob

from einops import rearrange

/mnt/disks/disk_main/anaconda3/envs/tune_a_video/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# controlnet = UNet3DConditionModel(64, controlnet_hint_channels=3, cross_attention_dim=768)
controlnet = UNet3DConditionModel.from_pretrained_2d("../../diffusers_controlnet/models/control_sd15_openpose/controlnet").half().cuda()

The config attributes {'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'time_cond_proj_dim': None, 'time_embedding_type': 'positional', 'timestep_post_act': None} were passed to UNet3DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [8]:
unet = UNet3DConditionModel.from_pretrained_2d("../../diffusers_controlnet/models/control_sd15_openpose/unet").half().cuda()

The config attributes {'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'time_cond_proj_dim': None, 'time_embedding_type': 'positional', 'timestep_post_act': None} were passed to UNet3DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [14]:
unet.enable_xformers_memory_efficient_attention()
controlnet.enable_xformers_memory_efficient_attention()

In [19]:
sample = torch.randn(1, 4, 3, 64, 64).half().cuda()
timestep = 1
encoder_hidden_states = torch.randn(1, 320, 768).half().cuda()
hint = torch.randn(1, 3, 3, 512, 512).half().cuda()
res = controlnet(sample=sample, timestep=timestep, encoder_hidden_states=encoder_hidden_states, controlnet_hint=hint)

In [17]:
res[0].shape

torch.Size([1, 320, 3, 64, 64])

In [20]:
res_unet = unet(sample=sample, timestep=timestep, encoder_hidden_states=encoder_hidden_states, control=res)

In [21]:
res_unet

UNet3DConditionOutput(sample=tensor([[[[[-5.6738e-01, -2.0972e-01, -1.1401e-01,  ..., -2.2107e-01,
            -2.3206e-01,  5.5957e-01],
           [ 7.4219e-02, -1.4502e-01, -1.2405e-02,  ...,  6.7578e-01,
            -9.8938e-02, -2.5610e-01],
           [ 1.9006e-01,  2.8198e-01, -4.5361e-01,  ..., -4.6704e-01,
            -7.4646e-02, -3.2501e-02],
           ...,
           [ 1.1879e-02, -2.2571e-01,  3.6841e-01,  ..., -2.5732e-01,
            -4.9805e-02,  7.1167e-02],
           [-2.7075e-01, -4.4165e-01,  5.9619e-01,  ..., -4.5581e-01,
            -5.6702e-02,  4.9976e-01],
           [ 2.9810e-01, -4.2383e-01,  5.5811e-01,  ...,  7.2876e-02,
             4.9512e-01, -5.1904e-01]],

          [[-4.5996e-01,  8.6035e-01, -5.1392e-02,  ...,  5.5420e-01,
             1.0242e-01, -4.8438e-01],
           [-4.9878e-01, -6.3904e-02, -2.7637e-01,  ...,  3.9612e-02,
            -2.7417e-01,  9.7949e-01],
           [ 5.1953e-01,  2.4216e-02,  1.0608e-01,  ..., -3.3447e-01,
           